In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import pickle
import sys
from tqdm import tqdm

from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch import cuda, device

from torchvision import transforms


from avalanche.benchmarks.classic import SplitCIFAR110
from avalanche.models import SimpleCNN
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    bwt_metrics,
    forgetting_metrics,
)

sys.path.append("../base_code/")

from base_code.constants import DATASETS_PATH, SAVED_METRICS_PATH
from base_code.seed import set_seed
from base_code.training import MWUNV1


/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
TORCH_DEVICE = device("cuda" if cuda.is_available() else "cpu")
N_EXPERIENCES = 11
EXPERIMENT_SEED = 1234
METRICS_SUBFOLDER = "cifar110"
METRICS_FILENAME = f"{METRICS_SUBFOLDER}/proposal_3.pkl"
set_seed(EXPERIMENT_SEED)


In [4]:
scenario = SplitCIFAR110(N_EXPERIENCES, seed=EXPERIMENT_SEED, dataset_root_cifar10=DATASETS_PATH, dataset_root_cifar100=DATASETS_PATH)

train_stream = scenario.train_stream
test_stream = scenario.test_stream


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [5]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    
    loggers=[InteractiveLogger()]
)

model = SimpleCNN(num_classes=scenario.n_classes).to(TORCH_DEVICE)
optimizer = SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=0.0005)
criterion = CrossEntropyLoss().to(TORCH_DEVICE)
strategy = MWUNV1(
    model,
    optimizer,
    criterion,
    eps=0.001,
    lambda_e=10.0,
    lambda_f=10.0,
    train_epochs=5,
    train_mb_size=128,
    evaluator=eval_plugin,
    eval_mb_size=128,
    device=TORCH_DEVICE,
)


In [6]:
for experience in tqdm(train_stream):
    strategy.train(experience)
    strategy.eval(test_stream)


  0%|          | 0/11 [00:00<?, ?it/s]

-- >> Start of training phase << --
100%|██████████| 391/391 [01:23<00:00,  4.70it/s]
Epoch 0 ended.
100%|██████████| 391/391 [01:23<00:00,  4.69it/s]
Epoch 1 ended.
100%|██████████| 391/391 [01:22<00:00,  4.72it/s]
Epoch 2 ended.
100%|██████████| 391/391 [01:22<00:00,  4.72it/s]
Epoch 3 ended.
100%|██████████| 391/391 [01:23<00:00,  4.71it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:04<00:00, 16.90it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.2331
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 8/8 [00:00<00:00, 15.14it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.0000
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████| 8/8 [00:00<00:00, 16.87it/s]
> Eval on exp

  9%|▉         | 1/11 [07:04<1:10:46, 424.67s/it]


> Eval on experience 10 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp010 = 0.0000
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = 0.0000
	StreamForgetting/eval_phase/test_stream = 0.0000
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.1166
-- >> Start of training phase << --
100%|██████████| 40/40 [00:08<00:00,  4.79it/s]
Epoch 0 ended.
100%|██████████| 40/40 [00:08<00:00,  4.76it/s]
Epoch 1 ended.
100%|██████████| 40/40 [00:08<00:00,  4.73it/s]
Epoch 2 ended.
100%|██████████| 40/40 [00:08<00:00,  4.69it/s]
Epoch 3 ended.
100%|██████████| 40/40 [00:08<00:00,  4.72it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:04<00:00, 16.97it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.2331
	ExperienceForgetting/eval_phase/test_stream/Task000/Ex

 18%|█▊        | 2/11 [07:56<30:46, 205.20s/it]  

...


> Eval on experience 10 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp010 = 0.0000
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.1213
	StreamForgetting/eval_phase/test_stream = 0.1213
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.0050
-- >> Start of training phase << --
100%|██████████| 40/40 [00:08<00:00,  4.68it/s]
Epoch 0 ended.
100%|██████████| 40/40 [00:08<00:00,  4.71it/s]
Epoch 1 ended.
100%|██████████| 40/40 [00:08<00:00,  4.70it/s]
Epoch 2 ended.
100%|██████████| 40/40 [00:08<00:00,  4.71it/s]
Epoch 3 ended.
100%|██████████| 40/40 [00:08<00:00,  4.71it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:04<00:00, 16.78it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.2331
	ExperienceForgetting/eval_phase/test_stream/Task000/E

 91%|█████████ | 10/11 [14:52<00:56, 56.71s/it]


> Eval on experience 10 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp010 = 0.0000
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.1189
	StreamForgetting/eval_phase/test_stream = 0.1189
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.0051
-- >> Start of training phase << --
100%|██████████| 40/40 [00:08<00:00,  4.62it/s]
Epoch 0 ended.
100%|██████████| 40/40 [00:08<00:00,  4.71it/s]
Epoch 1 ended.
100%|██████████| 40/40 [00:08<00:00,  4.69it/s]
Epoch 2 ended.
100%|██████████| 40/40 [00:08<00:00,  4.67it/s]
Epoch 3 ended.
100%|██████████| 40/40 [00:08<00:00,  4.70it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:04<00:00, 17.08it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.2331
	ExperienceForgetting/eval_phase/test_stream/Task000/E

100%|██████████| 11/11 [15:44<00:00, 85.88s/it]


> Eval on experience 10 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp010 = 0.1000
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.1172
	StreamForgetting/eval_phase/test_stream = 0.1172
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.0050


In [7]:
# save metrics
accuracies: dict[int, list[float]] = dict()
forgettings: dict[int, list[float]] = dict()
bwt: dict[int, list[float]] = dict()

for i in range(N_EXPERIENCES):
    filled_i = str(i).zfill(3)
    accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    forgettings[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceForgetting/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    bwt[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceBWT/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
        

accuracies["Overall"] = eval_plugin.get_all_metrics()[
    "Top1_Acc_Stream/eval_phase/test_stream/Task000"
][1]

pickle.dump({
    "accuracies": accuracies,
    "forgettings": forgettings,
    "bwt": bwt
}, open(SAVED_METRICS_PATH / METRICS_FILENAME, "wb"))


In [17]:
eval_plugin.get_all_metrics()['Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp010']


([1955, 2155, 2355, 2555, 2755, 2955, 3155, 3355, 3555, 3755, 3955],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1])